# Chargement du modèle GoogLeNet pré-entraîné

 charge le modèle **GoogLeNet** pré-entraîné sur le dataset ImageNet, en utilisant la bibliothèque PyTorch.

### Imports nécessaires

- **`torch`** : Bibliothèque principale de PyTorch pour le deep learning
- **`torch.nn`** : Module contenant les composants pour construire des réseaux de neurones
- **`torchvision.models`** : Module contenant des architectures de modèles pré-entraînés

In [28]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.googlenet(pretrained=True)



# Gel des paramètres d'un modèle
Ce code **gèle tous les paramètres** d'un modèle de réseau de neurones, empêchant leur mise à jour pendant l'entraînement.
Cette boucle parcourt **tous les paramètres** (poids et biais) du modèle et désactive le calcul des gradients pour chacun d'eux.


```python
    param.requires_grad = False
```
- `requires_grad` : Attribut qui indique si PyTorch doit calculer les gradients
- ` False` : Désactive le calcul des gradients pour ce paramètre

In [29]:
for param in model.parameters():
    param.requires_grad = False

# Remplacement du classificateur par un réseau personnalisé

## Description de la tâche

 **remplace la couche de classification finale** d'un modèle pré-entraîné (GoogLeNet) par un **réseau de neurones personnalisé** à plusieurs couches pour classifier 4 classes.

 crée un **classificateur multi-couches** pour adapter GoogLeNet à une tâche de classification à 4 classes.
#### 1. Définir le nombre de classes cibles

```python
num_classes = 4
```
- Nombre de catégories à prédire (par exemple : chien, chat, oiseau, poisson)

#### 2. Remplacer la couche finale

```python
model.fc = nn.Sequential(...)
```
- `model.fc` : La couche "fully connected" (dense) finale de GoogLeNet
- `nn.Sequential` : Conteneur qui empile plusieurs couches en séquence

#### 3. Architecture du nouveau classificateur

```python
nn.Linear(1024, 256)
```
- **Couche dense 1** : Transforme les 1024 features de GoogLeNet en 256 neurones
- `1024` : Dimension de sortie de GoogLeNet (features extraites)
- `256` : Nombre de neurones dans la couche cachée

```python
nn.ReLU()
```
- **Fonction d'activation** : Rectified Linear Unit
- Introduit la non-linéarité : `f(x) = max(0, x)`
- Permet au réseau d'apprendre des relations complexes

```python
nn.Dropout(0.5)
```
- **Régularisation** : Désactive aléatoirement 50% des neurones pendant l'entraînement
- Prévient le sur-apprentissage (overfitting)
- Rend le modèle plus robuste

```python
nn.Linear(256, num_classes)
```
- **Couche dense 2** : Transforme les 256 neurones en 4 sorties (une par classe)
- Produit les scores bruts (logits) pour chaque classe

```python
nn.Softmax(dim=1)
```
- **Fonction d'activation finale** : Convertit les logits en probabilités
- `dim=1` : Applique softmax sur la dimension des classes
- Résultat : Somme des probabilités = 1.0

## Architecture visuelle

```
Features GoogLeNet (1024) 
          ↓
    Linear (1024 → 256)
          ↓
       ReLU()
          ↓
    Dropout(50%)
          ↓
    Linear (256 → 4)
          ↓
    Softmax(dim=1)
          ↓
  Probabilités [0.1, 0.6, 0.2, 0.1]
```


In [30]:
num_classes = len(train_dataset.classes)
# print(len(train_dataset.classes))

model.fc = nn.Sequential(
    nn.Linear(1024, 256), 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes),
    nn.Softmax(dim=1)
)

In [31]:
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

# Configuration de l'entraînement du modèle


### 1. Learning Rate (Taux d'apprentissage)

```python
learning_rate = 0.001
```

- **Définition** : Contrôle la taille des pas lors de la mise à jour des poids

### 2. Loss Function (Fonction de perte)
- **Fonction** : Mesure l'erreur entre les prédictions et les vraies classes
- **CrossEntropyLoss** : Combinaison de Softmax + Negative Log Likelihood
- **Utilisation** : Classification multi-classes

### 3. Optimizer (Optimiseur)

```python
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
```

- **Adam** : Optimiseur adaptatif performant (combine momentum et RMSprop)
- **`model.fc.parameters()`** : Entraîne **uniquement** les nouvelles couches (le classificateur)
- **`lr=learning_rate`** : Applique le taux d'apprentissage défini

### Autres optimiseurs

```python
# SGD (plus simple, nécessite plus de réglages)
optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)

# AdamW (Adam avec weight decay)
optimizer = optim.AdamW(model.fc.parameters(), lr=0.001, weight_decay=0.01)

# RMSprop
optimizer = optim.RMSprop(model.fc.parameters(), lr=0.001)
```
### Autres loss functions

```python
# Pour classification binaire
loss_function = nn.BCEWithLogitsLoss()

# Pour régression
loss_function = nn.MSELoss()
```

In [41]:
# Determiner le learning rate et loss function et aussi l’optomizer pour le modele
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# print(device)
learning_rate  = 0.001   
loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)



In [33]:
import torch
import torchvision
from torch.utils.data import DataLoader

train_dataset = torch.load("../data/Data_Loaders/train_dataset.pt", weights_only=False)
val_dataset   = torch.load("../data/Data_Loaders/val_dataset.pt", weights_only=False)
test_dataset  = torch.load("../data/Data_Loaders/test_dataset.pt", weights_only=False)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


5204
1105
1108


In [43]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")


Epoch 1/5 - Loss: 1.0193, Accuracy: 74.33%
Epoch 2/5 - Loss: 0.8458, Accuracy: 90.97%
Epoch 3/5 - Loss: 0.8193, Accuracy: 93.27%
Epoch 4/5 - Loss: 0.8076, Accuracy: 94.16%
Epoch 5/5 - Loss: 0.8031, Accuracy: 94.64%


In [44]:
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

val_loss, val_acc = evaluate(model, val_loader, loss_function)
test_loss, test_acc = evaluate(model, test_loader, loss_function)
train_loss,train_acc=evaluate(model,train_loader,loss_function)

print(f"Validation : Loss = {val_loss:.4f}, Accuracy = {val_acc:.2f}%")
print(f"Test       : Loss = {test_loss:.4f}, Accuracy = {test_acc:.2f}%")
print(f"train       : Loss = {train_loss:.4f}, Accuracy = {train_acc:.2f}%")



Validation : Loss = 0.7670, Accuracy = 97.92%
Test       : Loss = 0.7634, Accuracy = 98.38%
train       : Loss = 0.7693, Accuracy = 97.77%
